In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gym openpyxl

In [3]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Chatbot deva Ai agent0/dummy data.xlsx')
# df = pd.read_excel('/content/drive/MyDrive/Copy of dummy data.xlsx')

In [4]:
df.count()

,0
User Input,53
Bot Response,53
Intent,53


In [5]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [8]:
import numpy as np
from gym import Env, spaces

In [14]:
class EnhancedChatbotEnv(Env):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.current_idx = 0
        self.session_history = []
        self.action_space = spaces.Discrete(len(data['Bot Response'].unique()))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(768 + 100,), dtype=np.float32)

    def reset(self):
        self.current_idx = 0
        state = self._get_state(self.current_idx)
        return state

    def step(self, action):
      # Get expected response
      expected_action = self.data['Bot Response'].iloc[self.current_idx]

      # Calculate reward (less harsh)
      reward = 1 if action == expected_action else 0  # or use your own feedback logic

      # Update session history before incrementing index
      self.session_history.append(self.data['User Input'].iloc[self.current_idx])

      # Increment index and loop if needed
      self.current_idx += 1
      if self.current_idx >= len(self.data):
          self.current_idx = 0
          done = True  # End episode after one pass through the dataset
      else:
          done = False

      # Get next state
      next_state = self._get_state(self.current_idx)
      return next_state, reward, done, {}

    def _get_state(self, idx):
        # Encode user input with BERT
        user_input = self.data['User Input'].iloc[idx]
        state_embedding = encode_text(user_input)
        # Encode session history (simplified: average of last 10 inputs)
        history_embedding = np.zeros(100)
        if len(self.session_history) > 0:
            recent_history = self.session_history[-10:]
            history_embeddings = [encode_text(text) for text in recent_history]
            history_embedding = np.mean(history_embeddings, axis=0)[:100]  # Truncate for demo
        # Concatenate state and history
        full_state = np.concatenate([state_embedding, history_embedding])
        return full_state

env = EnhancedChatbotEnv(df)

In [10]:
!pip install stable_baselines3

In [11]:
!pip install stable_baselines3

In [12]:
!pip install shimmy

In [17]:
import torch
print(torch.__version__)

2.6.0+cu124


In [18]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [20]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [16]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.get_device_name(0))  # Should show "Tesla T4" or similar

False


In [15]:
from stable_baselines3 import PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("enhanced_chatbot_rl_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [ ]:
from fastapi import FastAPI, Request
import numpy as np
from stable_baselines3 import PPO

app = FastAPI()
model = PPO.load("enhanced_chatbot_rl_model")
session_histories = {}  # Store session history per user

@app.post("/chat")
async def chat(request: Request):
    data = await request.json()
    user_id = data.get('user_id', 'default')
    user_input = data['user_input']
    # Encode user input and session history
    state_embedding = encode_text(user_input)
    history = session_histories.get(user_id, [])
    history_embedding = np.zeros(100)
    if history:
        recent_history = history[-10:]
        history_embeddings = [encode_text(text) for text in recent_history]
        history_embedding = np.mean(history_embeddings, axis=0)[:100]
    full_state = np.concatenate([state_embedding, history_embedding])
    # Predict action
    action, _ = model.predict(full_state)
    # Map action to response
    response = df['bot_response'].iloc[action]
    # Update session history
    session_histories[user_id] = history + [user_input]
    return {"response": response, "action_id": int(action)}

@app.post("/feedback")
async def feedback(request: Request):
    data = await request.json()
    user_id = data['user_id']
    action_id = data['action_id']
    helpful = data['helpful']  # True/False or 1/-1
    # Here, you can log feedback for later RL retraining
    # For now, just acknowledge
    return {"status": "feedback received"}